In [ ]:
#Load the required libraries

import cv2
import numpy as np
from skimage.filters import threshold_local

In [ ]:
#read the image 
image = cv2.imread('images/bcard.jpeg')

#Find the dimensions of the image
(h, w, d) = image.shape
print("width={}, height={}, depth={}".format(w, h, d))


In [ ]:
#Resize the image if required
r = 500.0/w
dim = (500, int(h*r))
resized = cv2.resize(image, dim)


In [ ]:
#Conver the image to gray scale
gray = cv2.cvtColor(resized,cv2.COLOR_BGR2GRAY)

#Apply Gaussian blur
gray = cv2.GaussianBlur(gray,(5,5),0)

#Detect the edges in the image
edged = cv2.Canny(gray, 75, 200)

#Show the image after and before edge detection
print("STEP 1: Edge Detection")
cv2.imshow("Image", resized)
cv2.imshow("Edged", edged)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
#Copy the image to a new variable
edged_copy = edged.copy()

#Find the contours in the image
image, cnts, hierarchy = cv2.findContours(edged_copy, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

#Sort the contours by contour area and get the top 5 contours
cnts = sorted(cnts,key = cv2.contourArea, reverse=True)[:5]

#Loop through the contours and get an approximate shape with four corners
for c in cnts:
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02*peri, True)
    
    if len(approx) == 4:
        screenCnt = approx
        break

print("STEP 2: Find contours of paper")
#Draw the contours and show the image
cv2.drawContours(resized, [screenCnt],-1, (0, 255, 0), 2)
cv2.imshow("Outline", resized)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
#Define order points functions to get the corners of the document in order
#The order of points is top left, top right,bottom right and bottom left
def order_pts(pts):
    rect = np.zeros((4,2), dtype = "float32")
    
    s= pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    
    return rect

In [ ]:
#Define a function to Apply perspective transform to get the top down view (hawk eye view) of the document
#Get the max width and max height of the four sides of the quadrilateral
#define destination points as shown below from the max height and max width
def four_point_transform(image, pts):
    rect = order_pts(pts)
    (tl, tr, br, bl) = rect
    
    #Get max width
    widthA = np.sqrt(((br[0] - bl[0])**2) + ((br[1] - bl[1])**2))
    widthB = np.sqrt(((tl[0] - tr[0])**2) + ((tl[1] - tr[1])**2))
    maxWidth = max(int(widthA), int(widthB))
    
    #Get max height
    heightA = np.sqrt(((tr[0] - br[0])**2) + ((tr[1] - br[1])**2))
    heightB = np.sqrt(((tl[0] - bl[0])**2) + ((tl[1] - bl[1])**2))
    maxHeight = max(int(heightA), int(heightB))
    
    #Define destination points frommax height and max width
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]
    ], dtype = "float32")
    
    #Apply perspective transforms using opencv functions
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
    
    #Return the warped image
    return warped

In [ ]:
#Warp the image to get a hawkeye view
warped = four_point_transform(resized, screenCnt.reshape(4,2))

#Convert the warped image to gray and apply adaptive thresholding to get a scanned document feel
warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
T = threshold_local(warped, 11,offset = 10, method = "gaussian")
warped = (warped > T).astype("uint8")*255

#Show the output
print("STEP 3:Apply perspective transform")
cv2.imshow("Original", resized)
cv2.imshow("Scanned", warped)
cv2.waitKey(0)
cv2.destroyAllWindows